## Procesamiento de Lenguaje Natural

*MINI-TASK \#2* 

# ***Corrector ortográfico de P. Norvig***

### **Equipo:**

- Burruel Durán Luis Andrés
- Giottonini Herrera Enrique Alejandro
- Villalba Miranda Jesús Abraham

**Fuentes**
* [How to Write a Spelling Corrector (Peter Norvig) ](https://norvig.com/spell-correct.html)
---

## **I. Introducción**

En el mundo en el que siempre estamos redactando información, séase mediante la generación de documentos o por la comunicación por medio de mensajes, una de las características más sobresalientes de los editores de texto que utilizamos, es la capacidad de corregir aquellas palabras que escribimos incorrectamente.

[Corrector ortográfico ](https://es.wikipedia.org/wiki/Corrector_ortogr%C3%A1fico#:~:text=Un%20corrector%20ortogr%C3%A1fico%20es%2C%20en,al%20usuario%20en%20su%20escritura.)
 > Un *corrector ortográfico* es una aplicación de software que se utiliza para analizar textos con el fin de detectar y, de forma automática o manual, corregir faltas ortográficas ayudando al usuario en su escritura.

### ¿Cómo funciona un corrector ortográfico?
Un corrector ortográfico sigue el siguiente proceso:
 1. **Identifica la palabra incorrecta**
    
    Identificamos que una palabra es incorrecta, es decir, no se encuentra dentro de nuestro vocabulario.

 2. **Se calculan las palabras a 1, 2 o 3 de distancia**
    
    Para esto se utiliza el algoritmo de **distancia mínima de edición**, lo que nos ayuda a tener una noción de similaridad entre dos palabras. 
    
    La **distancia mínima de edición** entre dos palabras la podemos definir como el mínimo número de operaciones de edición para transformar una cadena de caracteres (*source*) en otra (*target*).

    Las operaciones de edición pueden ser inserción, eliminación y remplazo de un carácter. A estas operaciones se les asigna un peso y basado en este, se obtiene la distancia.

 3. **Se filtran los posibles candidatos**
    
    De las palabras obtenidas en el paso anterior, se filtran los posibles candidatos de manera que se encuentren dentro de nuestro vocabulario.
    
 4. **Se calcula el más probable en funcion del contexto**

    Intentamos encontrar la corrección *c*, de todos los candidatos, de forma que maximize la probabilidad de que *c* es la corrección (palabra) correcta, dada la palabra incorrecta original. Para obtener dicha corrección, nos basamos en nuestro corpus.

Durante el resto del documento, explicaremos como funciona nuestra implementación de un corrector ortográfico basada en la libreta realizada por *P. Norvig*. Las secciones las podemos dividir en: II. ¿Cómo funciona?; en donde explicaremos como implementamos el corrector; III. Evaluación y IV. Conclusiones.

## **II. ¿Cómo funciona?**

Comenzamos por importar las librerias que utilizaremos


In [2]:
import re
from collections import Counter

Al realizar un corrector ortográfico, estamos intentando encontrar la corrección $c$, dentro de todos los posibles candidatos, de tal forma que maximize la probabilidad de que $c$ es la corrección correcta, dada la palabra original $w$.

$$argmax_{c \in candidates} P(c|w)$$

Por el teorema de Bayes, esta expresión es equivalente a

$$argmax_{c \in candidates} P(c) \dfrac{P(w|c)}{P(w)}$$

Como $P(w)$ es la misma para cualquier candidato $c$, podemos expresarlo de la siguiente manera:

$$argmax_{c \in candidates} P(c) P(w|c)$$

Esta expresión la podemos separar en cuatro partes:

### `1) Selection Mechanism`

---

Elegimos el candidato con mayor probabilidad combinada ($argmax$), esto lo vemos en la función `correccion`, con la palabra reservada `max` y el argumento `key`. 

La función `max` regresa el elemento más grande de un conjunto de elementos y `key` recibe una función con la cual los elementos son comparados.

```python
def correction(word):
    return max(candidates(word), key=P)
```

### `2) Candidate Model`

---

Generamos un conjunto de candidatos a partir de la palabra. Definimos a la edición simple de una cadena de caracteres como la eliminación, sustitución o inserción de algún caracter en la cadena original. La  función edits1  dada una palabra regresa el conjunto de todas las cadenas de caracteres que se pueden obtener al realizar alguna de las tres operaciones previamente mencionadas en la palabra dada.

Por otro lado, la función edits2 obtiene una palabra, y regresa el conjunto de todas las cadenas que se pueden obtener al realizar dos operaciones en dicha palabra.

In [3]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmñnopqrstuvwxyzáéíóú'
    splits     = [(word[:i], word[i:])    for i in range(len(word)+1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in (set(letters)-set(R[0]))]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes+replaces+inserts)

def edits2(word): return set(e2 for e1 in edits1(word) for e2 in edits1(e1))-edits1(word)-set(word)


Al considerar textos en español, nuestro conjunto de letras ("letters") posee ahora 32 símbolos, esto es, las 26 que comparte con el alfabeto inglés, más las cinco vocales con acento, mas el caracter "ñ". Así, con una cadena de caracteres de longitud n, podemos obtener n cadenas al eliminar un simbolo de la cadena original, 31n al realizar reemplazos y 32(n+1)-n al insertar símbolos, esto es, existen en total 63n+32 cadenas que están a una distancia igual a 1 de la palabra original.

### `3) Language Model`: $P(c)$

---

A partir de un corpus $C$ podemos generar un diccionario sencillo $D$ que contenga todas las palabras separadas por espacios de $C$ y su frecuencia de aparición en $C$. Podemos definir tambien de forma sencilla la probabilidad de una palabra $P(w)$ como la proporción de veces que aparece en $C$.

$P(c)$, es decir, la probabilidad que $c$ aparezca en nuestro texto. Para esto contamos el número de veces que cada tipo aparece en nuestro vocabulario.

Para esta tarea, utilizamos como *corpus* el libro de *Don Quijote de la Mancha* de *Miguel de Cervantes Saavedra* obtenido de [Project Gutenberg]("https://www.gutenberg.org/cache/epub/2000/pg2000.txt").

Para obtener nuestro vocabulario `WORDS` se utilizo un procesamiento sencillo de texto, en el que solo nos quedamos con caracteres alfanuméricos `re.findall(r'\w+', text.lower())` y los convertimos a minúscula `text.lower()`.

Una vez obtuvimos nuestro vocabulario, calculamos la frecuencia de cada tipo mediante la función `Counter`.



In [21]:
def words(text): return re.findall(r'\w+', text.lower())

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

WORDS = Counter(words(open('quijote.txt', encoding='utf-8').read()))

Para obtener la probabilidad $P(c)$, se creo una función `P(word, N)` que estima la probabilidad de cada palabra basada en su frecuencia. Donde `N` es la cantidad de palabras de nuestro corpus.

In [22]:
def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

In [23]:
print(f"Cardinalidad del vocabulario WORDS: {len(WORDS)}")
print(f"Cantidad de palabras del quijote: {sum(WORDS.values())}")
print(f"Palabras más comunes:")
for word in (WORDS.most_common(10)):
    print(word)
print(f"Probabilidad de la palabra 'que': {P('que')}")

Cardinalidad del vocabulario WORDS: 22942
Cantidad de palabras del quijote: 381226
Palabras más comunes:
('que', 20628)
('de', 18214)
('y', 18189)
('la', 10363)
('a', 9824)
('en', 8242)
('el', 8210)
('no', 6335)
('los', 4748)
('se', 4691)
Probabilidad de la palabra 'que': 0.0541096357541196


### `4) Error Model`: $P(w|c)$

---

La probabilidad de que la palabra $w$ se escriba en un texto cuando el autor quería decir $c$. Por ejemplo, $P(qur|que)$ es relativamente alta, pero $P(queeeexyz|que)$ tendría una probabilidad baja. 

La razón por la que aplicamos el teoréma de Bayes para seleccionar el elemento con propabilidad más grande $P(c|w)$ a una expresión de la forma $P(c)P(w|c)$ es porque queremos que nuestra corrección no solo dependa de la distancia entre el candidato $c$ y la palabra mal escrita $w$, sino tambien en que tan frecuente (en este caso probable) es haber querido escribir $c$.

Este *approach* tambien tiene sus problemas ya que  palabras inusuales en $C$ tendrán un $P(w)$ bajo. Por ejemplo, la palabra "Arrebol", que es un tecnisísmo sobre el efecto de la luz en las nubes, al escribirse erroneamente como "Arebl" es más probable que sea corregido como "árbol" en algún corpus de ciencias de plantas.

El otro problema es que palabras $w$ que no existan en $D$ tendrán una probabilidad de 0 y serán corregidas. Para estas dificultades son mitigadas con más datos, y actualmente con otros métodos más complejos.

El autor original del código no tenía ***data spellings errors***, es decir, datos con los cuales se pueda asignar reglas a la probabilidad de error. Por ejemplo, si los datos fueran palabras tecleadas por computadora entonces es más probable equivocarse en un carácter en las vecindades de esa tecla.

En este caso, se escogio que para el error de modelo la probabilidad de una palabra a una distancia edits 0 es infinitamente más probable que una palabra a distancia edits 1, y a su vez esta es infinitamente más probable que una palabra a distancia edits 2. Así que los candidatos están dados por orden de prioridad:

- 1. Palabras a edits 0 (palabra original) si está en el vocabulario.
- 2. Palabras a edits 1 en el vocabulario.
- 3. Palabras a edits 2 en el vocabulario.
- 4. Palabra $w$ aunque no esté en el vocabulario.

En este orden de candidatos se regresa aquel que maximize la probabilidad $P(c|w)$

In [24]:
def correction(word): return max(candidates(word), key=P)

def candidates(word): 
    return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]

## **III. Evaluación**

Hay 2 evaluaciones, primero pruebas de unidad para asegurar que las funciones regresen lo que esperamos que computen, y pruebas sobre el modelo, donde a partir de un dataset que contiene ejemplos de palabras escritas erroneamente y su corrección podemos evaluar como se desempeña nuestro corrector.

In [48]:
def unit_tests():
    assert correction("kijote") == "quijote"                # replace   
    assert correction("cabalo") == "caballo"                # insert
    assert correction("regalooo") == "regalo"               # delete
    assert correction("manana") == "mañana"                 # acentos
    assert correction("noche") == "noche"                   # conocida
    assert correction("computadora") == "computadora"       # desconocida para el Quijote
    assert words("ESTO es una PRUEba.") == ["esto", "es", "una", "prueba"]
    
    return 'unit_tests pass'

Para las pruebas de *"spelling"* esperamos como entrada una secuencia de datos de la forma (`right`, `wrong`) donde nosotros comprobamos que nuestra corrección/predicción sobre `wrong` sea `right`, hacemos lo mismo sobre $n$ datos de la misma forma y regresamos la proporción de aciertos y fallas, y la cantidad de tiempo de ejecución.

In [49]:
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.perf_counter()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w!= right:
            unknown += (right not in WORDS)
            if verbose:
                print(f'correction({wrong}) => {w} ({WORDS[w]}); expected {right} ({WORDS[right]})')

    dt = time.perf_counter() - start
    print(f"{good/n:.0%} of {n} correct ({unknown/n:.0%} unknown) at {n/dt:.0f} words per sec")

A partir de una secuencia de lineas podemos generar un dataset de pruebas para spelltest.

In [50]:
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(":") for line in lines)
            for wrong in wrongs.split()]

Esperamos que las pruebas unitarias pasen los `asserts` sin errores.

In [51]:
print(unit_tests())

unit_tests pass


In [29]:
spelltest(Testset(["caballo: cabayo cabalo caballio",]), verbose=True)

100% of 3 correct (0% unknown) at 12 words per sec


## **IV. Conclusiones**